In [206]:
import requests

import pandas as pd

import os
import apimoex
from moexalgo import Market, Ticker

from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [207]:
stocks = Market('stocks') # Раздел API с данныи по акциям, торгуемым в основное время
all_stocks = stocks.tickers() # DataFrame со всеми тикерами и данными по тикерам
tickers_list = all_stocks['ticker'].values # Формирование списка всех тикеров, торгуемых на мосбирже

In [208]:
open = {} # Словари для сбора данных
close = {}
high = {}
low = {}
value = {}
volume = {}

In [209]:
candle_time = 'hour'

In [210]:
INTERVAL_DICT = {
  'MIN': 1,
  'TEN': 10,
  'HOUR': 60,
  'DAY': 24,
  'WEEK': 7,
  'MONTH': 31,
  'QUARTAL': 4
}

In [211]:
def fetch_and_process_ticker(session, ticker):
  data = apimoex.get_market_candles(session, security=ticker, interval=INTERVAL_DICT[candle_time.upper()])
  df = pd.DataFrame(data)
  df.columns = ['time', 'open', 'close', 'high', 'low', 'value', 'volume']
  df.set_index('time', inplace=True)
  return {
      'ticker': ticker,
      'data': df
  }

In [212]:
# Создаем сессии requests.Session() и передаем их в функцию
def parallel_fetch(tickers_list):
  with requests.Session() as session:
    results = Parallel(n_jobs=-1)(
      delayed(fetch_and_process_ticker)(session, ticker) for ticker in tqdm(tickers_list)
    )
  return results

results = parallel_fetch(tickers_list)

for result in results:
  ticker = result['ticker']
  df = result['data']
  open[ticker] = df['open']
  close[ticker] = df['close']
  high[ticker] = df['high']
  low[ticker] = df['low']
  value[ticker] = df['value']
  volume[ticker] = df['volume']

  0%|          | 0/249 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))

In [213]:
close = pd.concat(close).reset_index(level=[0,1]).pivot_table(index='time', columns='level_0').droplevel(0, axis=1)
open = pd.concat(open).reset_index(level=[0,1]).pivot_table(index='time', columns='level_0').droplevel(0, axis=1)
high = pd.concat(high).reset_index(level=[0,1]).pivot_table(index='time', columns='level_0').droplevel(0, axis=1)
low = pd.concat(low).reset_index(level=[0,1]).pivot_table(index='time', columns='level_0').droplevel(0, axis=1)
value = pd.concat(value).reset_index(level=[0,1]).pivot_table(index='time', columns='level_0').droplevel(0, axis=1)
volume = pd.concat(volume).reset_index(level=[0,1]).pivot_table(index='time', columns='level_0').droplevel(0, axis=1)

ValueError: No objects to concatenate

In [214]:
close.to_csv(f'data/{candle_time}/close.csv')
open.to_csv(f'data/{candle_time}/open.csv')
high.to_csv(f'data/{candle_time}/high.csv')
low.to_csv(f'data/{candle_time}/low.csv')
volume.to_csv(f'data/{candle_time}/volume.csv')
value.to_csv(f'data/{candle_time}/value.csv')

AttributeError: 'dict' object has no attribute 'to_csv'